In [1]:
# Import only the packages necessary to check if FEFLOW and ifm_contrib is present
import sys
import os

# Check the current operating system
if sys.platform == 'win32':  # For Windows use \\, for Linux use /
    sys.path.append('C:\\Program Files\\DHI\\2025\\FEFLOW 10.0\\python')
    os.environ['FEFLOW10_ROOT'] = 'C:\\Program Files\\DHI\\2025\\FEFLOW 10.0\\'
    os.environ['FEFLOW_KERNEL_VERSION'] = '10'  # Modify for your use e.g. FEFLOW 7.4 would be 74
elif sys.platform == 'linux':
    sys.path.append('/opt/feflow/10.0/python/')
    os.environ['FEFLOW10_ROOT'] = '/opt/feflow/10.0/'
    os.environ['FEFLOW_KERNEL_VERSION'] = '10'  # Modify for your use e.g. FEFLOW 7.4 would be 74
else:
    sys.exit("Unsupported operating system.")
# Try to import the ifm package
try:
    from ifm import Enum
    import ifm_contrib as ifm
except ModuleNotFoundError:
    sys.exit("ifm_contrib could not be imported.")
# Check which version of FEFLOW is being used
if ifm.getKernelVersion() < 8000:
    sys.exit("This script is designed for FEFLOW version 8 and higher.")

# Import the rest of the required packages
import pandas as pd
import matplotlib.pyplot as plt
import time

In [2]:
# Define the location of the FEFLOW FEM file
INPUT_FEM = os.path.join(os.path.join(os.getcwd(), "fem"), "MonteCarlo.fem")
# Create the variable to load the FEM file
doc = None
loaded = False
# Create a variable to track the number of reconnection attempts
reconnection_attempts = 0

# Attempt to load the FEM file
while not loaded:
    try:
        # Load the FEFLOW model
        doc = ifm.loadDocument(INPUT_FEM)
        # Set the boolean to true for success
        loaded = True
    except ConnectionError:
        # Print an error message
        print("Failed to establish a connection to the FEFLOW license server.")
        # Check how many reconnection attempts have been made
        if reconnection_attempts > 3:
            sys.exit(f"Failed to establish a connection to the FEFLOW license server after {reconnection_attempts} attempts.")
        else:
            # Wait and retry
            time.sleep(300)
            reconnection_attempts += 1
    except FileNotFoundError:
        # Exit the program
        sys.exit("Failed to find the specified FEFLOW model.")

In [3]:
# Get and display the parameter lookup table
lookup_table = doc.getLookupTable()
lookup_table.getMaterials()

{'Sandstone': 1, 'Basalt': 2}

In [4]:
# Get the ID of a material if you don't want to use the table
mat_id = lookup_table.findMaterial("Sandstone")
print(mat_id)

1


In [5]:
# Add a material
lookup_table.addMaterial("Shale")
lookup_table.getMaterials()

{'Sandstone': 1, 'Basalt': 2, 'Shale': 3}

In [6]:
# Rename a material
lookup_table.setMaterialName(lookup_table.findMaterial("Shale"), "Fractured shale")
lookup_table.getMaterials()

{'Sandstone': 1, 'Basalt': 2, 'Fractured shale': 3}

In [7]:
# Remove a material
lookup_table.removeMaterial(lookup_table.findMaterial("Fractured shale"))
lookup_table.getMaterials()

{'Sandstone': 1, 'Basalt': 2}

In [8]:
# First, before we change parameter values of materials, always check if the material has the parameter
print(f"Sandstone has conductivity: {lookup_table.hasProperty(lookup_table.findMaterial("Sandstone"), Enum.P_CONDX)}")
print(f"Sandstone has porosity: {lookup_table.hasProperty(lookup_table.findMaterial("Sandstone"), Enum.P_UPORO)}")
# We can also get the entire parameter values list for a material with:
lookup_table.getProperties(lookup_table.findMaterial("Sandstone"))

Sandstone has conductivity: True
Sandstone has porosity: False


{101: 1.0}

In [9]:
# Set the parameter values of a material
lookup_table.setPropertyValue(lookup_table.findMaterial("Sandstone"), Enum.P_CONDX, 5e-1)
lookup_table.setPropertyValue(lookup_table.findMaterial("Basalt"), Enum.P_CONDX, 2e-2)
lookup_table.setPropertyValue(lookup_table.findMaterial("Sandstone"), Enum.P_CONDY, 5e-1)
lookup_table.setPropertyValue(lookup_table.findMaterial("Basalt"), Enum.P_CONDY, 2e-2)
lookup_table.setPropertyValue(lookup_table.findMaterial("Sandstone"), Enum.P_CONDZ, 5e-2)
lookup_table.setPropertyValue(lookup_table.findMaterial("Basalt"), Enum.P_CONDZ, 5e-4)
lookup_table.setPropertyValue(lookup_table.findMaterial("Sandstone"), Enum.P_UPORO, 1e-1)
lookup_table.setPropertyValue(lookup_table.findMaterial("Basalt"), Enum.P_UPORO, 2e-1)
# And subsequently view the material parameter value
lookup_table.getPropertyValue(lookup_table.findMaterial("Sandstone"), Enum.P_CONDX)
# We can also check everything that been assigned to this material
lookup_table.getProperties(lookup_table.findMaterial("Sandstone"))

{101: 0.5, 103: 0.5, 105: 0.05, 117: 0.1}

In [10]:
# Close the FEM file
doc.closeDocument()